In [84]:
#Taken from "A Heuristic Approach to Finding Diverse Short Paths" (Caleb Voss, Mark Moll, and Lydia E. Kavraki)

# Imports
import networkx as nx
import numpy as np
import cvxpy as cp
import math
import osmnx as  ox

def diverseShortestPaths(graph, source, target, numberOfPaths, branchingFactor, ballRadius):
    # FIFO Queue  (append & pop(0))
    shortestPath_graph_pairs = []
    # A set of k-diverse SPs
    shortestPaths = []
    
    shortestPath = getShortestPath(graph, source, target)
        
    if (len(shortestPath) != 0 ):
        shortestPath_graph_pairs.append((shortestPath, graph))
        shortestPaths.append(shortestPath)
        
    while (len(shortestPath_graph_pairs) != 0):
        pair = shortestPath_graph_pairs.pop(0)
        shortestPath = pair[0]
        graph = pair[1]
        
        for i in range(branchingFactor):
            sampledEdge = edgeSampling(graph, shortestPath)
            
            nodes, edges = ox.graph_to_gdfs(graph)
            
            newEdges = getNewEdges(edges, sampledEdge, ballRadius)
            
            newGraph = ox.graph_from_gdfs(nodes, newEdges)
            
            newShortestPath = getShortestPath(newGraph, source, target)
            
            if (len(shortestPath) != 0):
                shortestPath_graph_pairs.append((shortestPath, newGraph))
            
            #TODO
            if (True): #NO CRITERIA, FOR NOW. (ACCEPT ALL)
                shortestPaths.append(newShortestPath)
                
            if (len(shortestPaths) == numberOfPaths):
                return shortestPaths
            
    return shortestPaths


def edgeSampling(graph, path):
    pathWeight = getPathWeight(path, graph)
    probability_distribution = []
    edges = []
    
    for i in range(len(path) - 1):
        j = i + 1
        source = path[i]
        target = path[j]
        edges.append((source, target))
        
        edgeWeight = graph[source][target][0]['weight']
        
        probability_distribution.append(edgeWeight / pathWeight)
        
    edge = np.choice(edges, 1,
              p=probability_distribution, replace=False)
    
    return edge    
    

def getNewEdges(oldEdges, sampledEdge, ballRadius):
    distances = oldEdges.apply(getMinDistance, args=(sampledEdge, ), axis=1)
    mask = distances > ballRadius
    newEdges = oldEdges[mask]
    
    return newEdges


# only intersects if both have the same nnode, so no need to consider other intersection cases
def getMinDistance(edge1, edge2):
    points1 = list(edge1['geometry'].coords)
    points2 = list(edge2['geometry'].coords)
    minDistance = float('inf')
    
    for i in range(len(points1)):
        point1 = points1[i]
        for j in range(len(points2)):
            point2 = points2[j]
            distance = getDistance(point1, point2)
            if (distance < minDistance):
                minDistance = distance
    
    return minDistance


def getDistance(point1, point2):
    return math.sqrt((point1[0] - point2[0])**2 + (point1[1] - point2[1])**2)


def getShortestPath(graph, source, target):
    try:
        shortestPath = nx.shortest_path(graph, source=source, target=target, weight="weight")
    except:
        shortestPath = []
        
    return shortestPath


def getPathWeight(path, graph):
    weights = 0
    for i in range(len(path) - 1):
        j = i + 1
        source = path[i]
        target = path[j]
        weights += graph[source][target][0]['weight']
        
    return weights
        

In [85]:
address = '30 Aldwych, London WC2B 4BG'
G = ox.graph_from_address(address, network_type="drive")

In [87]:
desired_path = [1696030874, 109753, 1617512815, 1707216637, 21392100, 109757, 1707216642, 25472888, 1707216646, 1678452728, 4879371166, 4421008555, 4421008566, 4034060018, 367102039, 4166662878, 26374229, 25378124, 107698, 6139961783, 107697, 282569739]

In [89]:
s = diverseShortestPaths(G, desired_path[0], desired_path[-1], 5, 10, 0.02)

KeyError: 'weight'

In [3]:
nodes, edges = ox.graph_to_gdfs(G)

In [94]:
edges[edges.index.get_level_values('u') == 104302 and edges.index.get_level_values('v') == 107844]

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [99]:
edges.loc[(104302, 107844)].iloc[0]

/var/folders/gw/jkn0nk5s0rdcwm1bz4hpz41h0000gn/T/ipykernel_11600/3452099121.py:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  edges.loc[(104302, 107844)].iloc[0]


osmid                                                 2644235
oneway                                                   True
name                                            Keppel Street
highway                                           residential
maxspeed                                               20 mph
reversed                                                False
length                                                 69.735
geometry    LINESTRING (-0.1303251 51.5203679, -0.1302613 ...
lanes                                                     NaN
ref                                                       NaN
tunnel                                                    NaN
access                                                    NaN
junction                                                  NaN
bridge                                                    NaN
Name: 0, dtype: object

In [14]:
# 0(pin):1611771529
# 1(pin):1611771530
# edges[edges.index]
#edges['geometry']

for i in range(len(edges.index)):
    z = edges.index[i]
    z1 = z[0]
    z2 = z[1]
    if z1 == 1611771529 and z2 == 1611771530:
        print(i)

926
927


In [22]:
points = list(edges.iloc[926]['geometry'].coords)

In [23]:
getDistance(points[0], points[1])

0.00043214342526299996

In [20]:
def getDistance(point1, point2):
    return math.sqrt((point1[0] - point2[0])**2 + (point1[1] - point2[1])**2)

In [58]:
# only intersects if both have the same nnode, so no need to consider other intersection cases
def getMinDistance(edge1, edge2):
    points1 = list(edge1['geometry'].coords)
    points2 = list(edge2['geometry'].coords)
    
    minDistance = float('inf')
    
    for i in range(len(points1)):
        point1 = points1[i]
        for j in range(len(points2)):
            point2 = points2[j]
            distance = getDistance(point1, point2)
            if (distance < minDistance):
                minDistance = distance
    
    return minDistance

In [59]:
edge1 = edges.iloc[0]
edge2 = edges.iloc[1]

getMinDistance(edge1, edge2)

0.0

In [66]:
def getNewEdges(oldEdges, sampledEdge, ballRadius):
    distances = oldEdges.apply(getMinDistance, args=(sampledEdge, ), axis=1)
    mask = distances > ballRadius
    newEdges = oldEdges[mask]
    
    return newEdges


In [81]:
newEdges = getNewEdges(edges, edge1, 0.02)

In [82]:
print(len(edges), len(newEdges))

1440 474
